In [ ]:
import pandas as pd


In [ ]:
movies = pd.read_csv("movies_cleaned.csv")

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 0)
classifier("I love this!")

In [ ]:
predictions = classifier(movies["Plot"][900])

In [ ]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

In [ ]:
for label in emotion_labels:
    movies[label] = 0.0

In [ ]:
predictions[0]

In [ ]:
for emotion in predictions[0]:
    print(emotion["score"])

In [ ]:
movies.at[5, "anger"] 

In [ ]:
movies["Plot"]

In [ ]:
import numpy as np


unique_id = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [ ]:
for i in range(10):
    unique_id.append(movies["unique_id"][i])
    sentences = movies["Plot"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [ ]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
unique_id = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(movies))):
    unique_id.append(movies["unique_id"][i])
    sentences = movies["Plot"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [ ]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["unique_id"] = unique_id

In [ ]:
movies = pd.merge(movies, emotions_df, on = "unique_id")

In [ ]:
movies.to_csv("movies_with_emotions.csv", index = False)